In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json

In [ ]:
def get_web_page(url):
    resp = requests.get(
    url = url,
    cookies = {'over18':'1'}  # 告知 Server 已回答滿18歲的問題
    )
    if resp.status_code != 200:
        print('Invalid url:',resp.url)
    else:
        return resp.text

In [38]:
def get_articles(dom,date):
    soup = BeautifulSoup(dom, 'html5lib')
    
    # 取得上一頁的連接
    paging_div = soup.find('div','btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href'] # 從0開始，1是指按鈕上頁
    
    articles = [] #儲存取得的文章資料
    divs = soup.find_all('div','r-ent')
    for d in divs:
        if d.find('div','date').text.strip() ==  date: #發文日期正確
            # 取得推文數
            push_count = 0
            push_str = d.find('div','nrec').text
            if push_str:
                try:
                    push_count = int(push_str) # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是' 爆' 或 x1, x2,...
                    # 若不是， 不做任何事， push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10
                        
            # 取得文章連接及標題
            if d.find('a'): #有超鏈接，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = d.find('div','author').text if d.find('div','author') else ''
                articles.append({
                    'title' : title,
                    'href' : href,
                    'push_count' : push_count,
                    'author' : author
                })
    return articles, prev_url

In [44]:
PTT_URL = 'https://www.ptt.cc'
current_page = get_web_page(PTT_URL + '/bbs/Headphone/index.html')

if current_page:
    
    articles = [] # 全部的今日文章
    
    # 今天日期，去掉開頭的 '0' 以符合PTT網站格式
    today = time.strftime('%m/%d').lstrip('0')
    
    # 目前頁面的今日文章
    current_articles,prev_url = get_articles(current_page,today)
    
    # 若目前頁面有今日文章則加入 articles, 並回到上一頁繼續尋找是否有今日文章
    while current_articles:
        articles += current_articles
        current_page = get_web_page(PTT_URL + prev_url)
        current_articles, prev_url = get_articles(current_page, today)
        
    # 儲存或處理文章資訊
    print('今天有', len(articles),'篇文章')
    threshold = 50
    print(' 熱門文章(> %d 推:):' % (threshold)) # 大於50的推薦熱門文章
    for a in articles:
        if int(a['push_count']) > threshold:
            print(a)
    # 關於 json.dump https://docs.python.org/zh-tw/3/library/json.html
    with open('Headphone.json','w', encoding = 'utf-8') as f:
        json.dump(articles, f , indent = 2 , sort_keys = True , ensure_ascii = False)

今天有 11 篇文章
 熱門文章(> 50 推:):
